## Download source code from github

In [1]:
!git clone https://github.com/google-deepmind/alphageometry

%cd ./alphageometry

!ls

Cloning into 'alphageometry'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 77 (delta 28), reused 65 (delta 19), pack-reused 0
Receiving objects: 100% (77/77), 1.05 MiB | 4.30 MiB/s, done.
Resolving deltas: 100% (28/28), done.
/content/alphageometry
alphageometry.py       defs.txt			   imo_ag_30.txt	 README.md
alphageometry_test.py  download.sh		   jgex_ag_231.txt	 requirements.in
ar.py		       examples.txt		   LICENSE		 requirements.txt
ar_test.py	       fig1.svg			   lm_inference.py	 rules.txt
beam_search.py	       geometry_150M_generate.gin  lm_inference_test.py  run.sh
CONTRIBUTING.md        geometry.py		   models.py		 run_tests.sh
ddar.py		       geometry_test.py		   numericals.py	 trace_back.py
ddar_test.py	       graph.py			   numericals_test.py	 trace_back_test.py
dd.py		       graph_test.py		   pretty.py		 transformer_layer.py
dd_test.py	       graph_utils.py		   problem.py


# Install VirtualEnv

In [2]:
!pip3 install virtualenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 8.4 MB/s eta 0:00:00


# Run instructions

## Installation is done in a virtual environment

In [3]:
%%shell

set -e
set -x

virtualenv -p python3 .
source ./bin/activate

pip install --require-hashes -r requirements.txt

+ virtualenv -p python3 .
created virtual environment CPython3.10.12.final.0-64 in 1306ms
  creator CPython3Posix(dest=/content/alphageometry, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.3.1, setuptools==69.0.2, wheel==0.42.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
+ source ./bin/activate
++ '[' ./bin/activate = /bin/bash ']'
++ deactivate nondestructive
++ unset -f pydoc
++ '[' -z '' ']'
++ '[' -z '' ']'
++ hash -r
++ '[' -z '' ']'
++ unset VIRTUAL_ENV
++ unset VIRTUAL_ENV_PROMPT
++ '[' '!' nondestructive = nondestructive ']'
++ VIRTUAL_ENV=/content/alphageometry
++ '[' linux-gnu = cygwin ']'
++ '[' linux-gnu = msys ']'
++ export VIRTUAL_ENV
++ _OLD_VIRTUAL_PATH=/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/us

## Download weights and vocabulary

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Vocabulary downloaded from https://bit.ly/alphageometry

In [5]:
%%shell

DATA="/content/drive/MyDrive/ag_ckpt_vocab"

ls $DATA

checkpoint_10999999  geometry.757.model  geometry.757.vocab


## Install meliad separately as it is not registered with pip

In [6]:
%%shell

MELIAD_PATH=meliad_lib/meliad
mkdir -p $MELIAD_PATH
git clone https://github.com/google-research/meliad $MELIAD_PATH
export PYTHONPATH=$PYTHONPATH:/content/alphageometry/$MELIAD_PATH

echo $PYTHONPATH

Cloning into 'meliad_lib/meliad'...
remote: Enumerating objects: 163, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 163 (delta 35), reused 28 (delta 28), pack-reused 110
Receiving objects: 100% (163/163), 1.98 MiB | 5.73 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/env/python:/content/meliad_lib/meliad


## Set up definitions and rules files

In [7]:
%%shell

DDAR_ARGS=(
  --defs_file=$(pwd)/defs.txt \
  --rules_file=$(pwd)/rules.txt \
);

## Lightweight values for the proof search parameters

In [8]:
%%shell

BATCH_SIZE=2
BEAM_SIZE=2
DEPTH=2

SEARCH_ARGS=(
  --beam_size=$BEAM_SIZE
  --search_depth=$DEPTH
)

## Define the flags relevant to the language model

In [9]:
%%shell

DATA="/content/drive/MyDrive/ag_ckpt_vocab"

MELIAD_PATH=meliad_lib/meliad
export PYTHONPATH=$PYTHONPATH:$MELIAD_PATH

LM_ARGS=(
  --ckpt_path=$DATA \
  --vocab_path=$DATA/geometry.757.model \
  --gin_search_paths=$MELIAD_PATH/transformer/configs \
  --gin_file=base_htrans.gin \
  --gin_file=size/medium_150M.gin \
  --gin_file=options/positions_t5.gin \
  --gin_file=options/lr_cosine_decay.gin \
  --gin_file=options/seq_1024_nocache.gin \
  --gin_file=geometry_150M_generate.gin \
  --gin_param=DecoderOnlyLanguageModelGenerate.output_token_losses=True \
  --gin_param=TransformerTaskConfig.batch_size=$BATCH_SIZE \
  --gin_param=TransformerTaskConfig.sequence_length=128 \
  --gin_param=Trainer.restore_state_variables=False
);

## Pass all tests in the prepared test suite

In [10]:
%%shell

export PYTHONPATH=$PYTHONPATH:$MELIAD_PATH

python problem_test.py

Running tests under Python 3.10.12: /usr/bin/python3
[ RUN      ] ProblemTest.test_orthocenter_no_translate
[       OK ] ProblemTest.test_orthocenter_no_translate
[ RUN      ] ProblemTest.test_orthocenter_translate
[       OK ] ProblemTest.test_orthocenter_translate
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


In [11]:
%%shell

python geometry_test.py

Running tests under Python 3.10.12: /usr/bin/python3
[ RUN      ] GeometryTest.test_bfs_for_equality_transitivity
[       OK ] GeometryTest.test_bfs_for_equality_transitivity
[ RUN      ] GeometryTest.test_merged_node_equivalence
[       OK ] GeometryTest.test_merged_node_equivalence
[ RUN      ] GeometryTest.test_merged_node_representative
[       OK ] GeometryTest.test_merged_node_representative
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK


In [12]:
%%shell

python graph_utils_test.py

Running tests under Python 3.10.12: /usr/bin/python3
[ RUN      ] GraphUtilsTest.test_comb2
[       OK ] GraphUtilsTest.test_comb2
[ RUN      ] GraphUtilsTest.test_comb3
[       OK ] GraphUtilsTest.test_comb3
[ RUN      ] GraphUtilsTest.test_comb4
[       OK ] GraphUtilsTest.test_comb4
[ RUN      ] GraphUtilsTest.test_cross
[       OK ] GraphUtilsTest.test_cross
[ RUN      ] GraphUtilsTest.test_perm2
[       OK ] GraphUtilsTest.test_perm2
[ RUN      ] GraphUtilsTest.test_perm3
[       OK ] GraphUtilsTest.test_perm3
[ RUN      ] GraphUtilsTest.test_perm4
[       OK ] GraphUtilsTest.test_perm4
----------------------------------------------------------------------
Ran 7 tests in 0.001s

OK


## Use a virtual framebuffer (Xvfb)
We need to run the code in a headless environment and require a graphical display for the TkAgg backend, we can use a virtual framebuffer like Xvfb. This creates a virtual display without a physical monitor.

### Install Xvfb

In [13]:
%%shell

sudo apt-get install xvfb

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings
  xfonts-utils xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings
  xfonts-utils xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 29 not upgraded.
Need to get 7,816 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 x11-xkb-utils amd64 7.7+5build4 [172 kB]
Get:5 http://archiv

### Run the script with Xvfb

This will simulate a display for the script to use.

In [14]:
%%shell

xvfb-run python numericals_test.py

Running tests under Python 3.10.12: /usr/bin/python3
[ RUN      ] NumericalTest.test_sketch_2l1c
[       OK ] NumericalTest.test_sketch_2l1c
[ RUN      ] NumericalTest.test_sketch_3peq
[       OK ] NumericalTest.test_sketch_3peq
[ RUN      ] NumericalTest.test_sketch_aline
[       OK ] NumericalTest.test_sketch_aline
[ RUN      ] NumericalTest.test_sketch_amirror
[       OK ] NumericalTest.test_sketch_amirror
[ RUN      ] NumericalTest.test_sketch_bisect
[       OK ] NumericalTest.test_sketch_bisect
[ RUN      ] NumericalTest.test_sketch_bline
[       OK ] NumericalTest.test_sketch_bline
[ RUN      ] NumericalTest.test_sketch_cc_tangent
[       OK ] NumericalTest.test_sketch_cc_tangent
[ RUN      ] NumericalTest.test_sketch_circle
[       OK ] NumericalTest.test_sketch_circle
[ RUN      ] NumericalTest.test_sketch_e5128
[       OK ] NumericalTest.test_sketch_e5128
[ RUN      ] NumericalTest.test_sketch_edia_quadrangle
[       OK ] NumericalTest.test_sketch_edia_quadrangle
[ RUN      ] 

In [15]:
%%shell

xvfb-run python graph_test.py

Running tests under Python 3.10.12: /usr/bin/python3
I0121 21:30:16.332235 136457653657600 graph.py:498] 
I0121 21:30:16.332433 136457653657600 graph.py:499] a b c = triangle a b c; h = orthocenter a b c; h1 = foot a b c; h2 = foot b c a; h3 = foot c a b; g1 g2 g3 g = centroid g1 g2 g3 g a b c; o = circle a b c ? coll h g o
[ RUN      ] GraphTest.test_build_graph_points
[       OK ] GraphTest.test_build_graph_points
[ RUN      ] GraphTest.test_build_graph_predicates
[       OK ] GraphTest.test_build_graph_predicates
[ RUN      ] GraphTest.test_enumerate_circles
[       OK ] GraphTest.test_enumerate_circles
[ RUN      ] GraphTest.test_enumerate_colls
[       OK ] GraphTest.test_enumerate_colls
[ RUN      ] GraphTest.test_enumerate_congs
[       OK ] GraphTest.test_enumerate_congs
[ RUN      ] GraphTest.test_enumerate_cyclics
[       OK ] GraphTest.test_enumerate_cyclics
[ RUN      ] GraphTest.test_enumerate_eqangles
[       OK ] GraphTest.test_enumerate_eqangles
[ RUN      ] GraphTest.t

In [16]:
%%shell

xvfb-run python dd_test.py

Running tests under Python 3.10.12: /usr/bin/python3
[ RUN      ] DDTest.test_imo_2022_p4_should_succeed
I0121 21:30:48.363947 140105309466624 graph.py:498] 
I0121 21:30:48.364224 140105309466624 graph.py:499] a b = segment a b; c = on_tline c a a b; d = on_tline d b b a; e = on_circle e c a, on_circle e d b; f = on_circle f c a, on_circle f d b; g = on_pline g e a b, on_circle g c a; h = on_pline h e a b, on_circle h d b; i = on_line i a g, on_line i b h; j = on_line j a f, on_line j g h; k = on_line k b f, on_line k g h ? cong i j i k
[       OK ] DDTest.test_imo_2022_p4_should_succeed
[ RUN      ] DDTest.test_incenter_excenter_should_fail
I0121 21:31:14.147334 140105309466624 graph.py:498] 
I0121 21:31:14.147497 140105309466624 graph.py:499] a b c = triangle a b c; d = incenter d a b c; e = excenter e a b c ? perp d c c e
[       OK ] DDTest.test_incenter_excenter_should_fail
----------------------------------------------------------------------
Ran 2 tests in 25.803s

OK


In [17]:
%%shell

xvfb-run python ar_test.py

Running tests under Python 3.10.12: /usr/bin/python3
[ RUN      ] ARTest.test_angle_table_equilateral_triangle
I0121 21:31:15.104806 139518998441984 graph.py:498] 
I0121 21:31:15.104988 139518998441984 graph.py:499] a b c = ieq_triangle ? cong a b a c
[       OK ] ARTest.test_angle_table_equilateral_triangle
[ RUN      ] ARTest.test_angle_table_inbisector_exbisector
I0121 21:31:15.116487 139518998441984 graph.py:498] 
I0121 21:31:15.116652 139518998441984 graph.py:499] a b c = triangle a b c; d = incenter d a b c; e = excenter e a b c ? perp d c c e
[       OK ] ARTest.test_angle_table_inbisector_exbisector
[ RUN      ] ARTest.test_generic_table_simple
[       OK ] ARTest.test_generic_table_simple
[ RUN      ] ARTest.test_incenter_excenter_touchpoints
I0121 21:31:15.134101 139518998441984 graph.py:498] 
I0121 21:31:15.134268 139518998441984 graph.py:499] a b c = triangle a b c; d1 d2 d3 d = incenter2 a b c; e1 e2 e3 e = excenter2 a b c ? perp d c c e
[       OK ] ARTest.test_incenter_e

In [18]:
%%shell

xvfb-run python ddar_test.py

Running tests under Python 3.10.12: /usr/bin/python3
[ RUN      ] DDARTest.test_incenter_excenter_should_succeed
I0121 21:31:16.133462 134280590761984 graph.py:498] 
I0121 21:31:16.133624 134280590761984 graph.py:499] a b c = triangle a b c; d = incenter d a b c; e = excenter e a b c ? perp d c c e
I0121 21:31:16.163600 134280590761984 ddar.py:60] Depth 1/1000 time = 0.014475345611572266
I0121 21:31:16.175985 134280590761984 ddar.py:60] Depth 2/1000 time = 0.011721372604370117
[       OK ] DDARTest.test_incenter_excenter_should_succeed
[ RUN      ] DDARTest.test_orthocenter_aux_should_succeed
I0121 21:31:16.176476 134280590761984 graph.py:498] 
I0121 21:31:16.176575 134280590761984 graph.py:499] a b c = triangle a b c; d = on_tline d b a c, on_tline d c a b; e = on_line e a c, on_line e b d ? perp a d b c
I0121 21:31:16.199577 134280590761984 ddar.py:60] Depth 1/1000 time = 0.01912212371826172
I0121 21:31:16.223515 134280590761984 ddar.py:60] Depth 2/1000 time = 0.023777484893798828
I0

In [19]:
%%shell

xvfb-run python trace_back_test.py

Running tests under Python 3.10.12: /usr/bin/python3
[ RUN      ] TracebackTest.test_orthocenter_dependency_difference
I0121 21:31:17.249560 139281541115904 graph.py:498] 
I0121 21:31:17.249751 139281541115904 graph.py:499] a b c = triangle a b c; d = on_tline d b a c, on_tline d c a b; e = on_line e a c, on_line e b d ? perp a d b c
I0121 21:31:17.296780 139281541115904 ddar.py:60] Depth 1/1000 time = 0.04069161415100098
I0121 21:31:17.321156 139281541115904 ddar.py:60] Depth 2/1000 time = 0.02416539192199707
I0121 21:31:17.351881 139281541115904 ddar.py:60] Depth 3/1000 time = 0.030548810958862305
[       OK ] TracebackTest.test_orthocenter_dependency_difference
----------------------------------------------------------------------
Ran 1 test in 0.109s

OK


### Install python modules

In [21]:
!pip install clu==0.0.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 2.3 MB/s eta 0:00:00
  Using cached ml_collections-0.1.1-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 29.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
tensorflow 2.15.0 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.3.2 which is incompatible.
tensorflow 2.15.0 requires numpy<2.0.0,>=1.23.5, but you have numpy 1.23.1 which is incompatible.
tf-nightly 2.16.0.dev20240119 requires

In [ ]:
!pip install seqio==0.0.18 

In [22]:
!pip install seqio-nightly==0.0.17.dev20231013

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.5/354.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.8/577.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 74.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 5.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of te

In [ ]:
!pip install t5==0.9.4

In [ ]:
!pip install flax==0.5.3

In [ ]:
!pip install jax==0.4.6

In [ ]:
!pip install jaxlib==0.4.6

### Install Tensorflow

In [23]:
!pip install tensorflow==2.13.0

  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1.post1-py2.py3-none-any.whl size=17283 sha256=f69454d80bd624cc3dfa3510dd93ae6925c423858de268bd0dcda3daf2b98372
  Stored in directory: /root/.cache/pip/wheels/f4/c8/0e/b79b08e45752491b9acfdbd69e8a609e8b2ed7640dda5a3e59
Successfully built tensorrt


In [ ]:
### Install TensorRT

In [ ]:
!python3 -m pip install --upgrade tensorrt

### Verify that TensorRT installation is working

In [24]:
import tensorrt
print(tensorrt.__version__)
assert tensorrt.Builder(tensorrt.Logger())

8.6.1


### Test alphageometry

In [26]:
%%shell

MELIAD_PATH=meliad_lib/meliad
export PYTHONPATH=$PYTHONPATH:/content/alphageometry/$MELIAD_PATH

echo $PYTHONPATH

CUDNN_PATH="/usr/local/lib/python3.10/dist-packages/nvidia/cudnn/"
TENSOR_LIBS="/usr/local/lib/python3.10/dist-packages/tensorrt_libs"
export LD_LIBRARY_PATH=$TENSOR_LIBS:$CUDNN_PATH/lib:$LD_LIBRARY_PATH

echo $LD_LIBRARY_PATH

xvfb-run python alphageometry_test.py

/env/python:/content/alphageometry/meliad_lib/meliad
2024-01-21 21:42:38.626466: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/alphageometry/alphageometry_test.py", line 21, in <module>
    import alphageometry
  File "/content/alphageometry/alphageometry.py", line 28, in <module>
    import lm_inference as lm
  File "/content/alphageometry/lm_inference.py", line 20, in <module>
    import models  # pylint: disable=unused-import
  File "/content/alphageometry/models.py", line 24, in <module>
    from transformer import models
  File "/content/alphageometry/meliad_lib/meliad/transformer/models.py", line 25, in <module>
    import  metrics_summary
  File "/content/alphageometry/meliad_lib/meliad/metrics_summary.py", line 20, in <module>
    from clu import metric_writers
  File "/usr/local/lib/python3.10/dist-packages/clu/metric_writers/__init__.py", line 53, in <module>
    from clu.met

CalledProcessError: Command '
MELIAD_PATH=meliad_lib/meliad
export PYTHONPATH=$PYTHONPATH:/content/alphageometry/$MELIAD_PATH

echo $PYTHONPATH

xvfb-run python alphageometry_test.py
' returned non-zero exit status 1.

In [ ]:
%%shell

MELIAD_PATH=meliad_lib/meliad
export PYTHONPATH=$PYTHONPATH:/content/alphageometry/$MELIAD_PATH

echo $PYTHONPATH

DATA="/content/drive/MyDrive/ag_ckpt_vocab"

echo $DATA

CUDNN_PATH="/usr/local/lib/python3.10/dist-packages/nvidia/cudnn/"
TENSOR_LIBS="/usr/local/lib/python3.10/dist-packages/tensorrt_libs"
export LD_LIBRARY_PATH=$TENSOR_LIBS:$CUDNN_PATH/lib:$LD_LIBRARY_PATH

echo $LD_LIBRARY_PATH

python lm_inference_test.py --meliad_path=$MELIAD_PATH --data_path=$DATA

## Run DDAR

In [ ]:
%%shell

MELIAD_PATH=meliad_lib/meliad
export PYTHONPATH=$PYTHONPATH:/content/alphageometry/$MELIAD_PATH

echo $PYTHONPATH

DATA="/content/drive/MyDrive/ag_ckpt_vocab"

echo $DATA

CUDNN_PATH="/usr/local/lib/python3.10/dist-packages/nvidia/cudnn/"
TENSOR_LIBS="/usr/local/lib/python3.10/dist-packages/tensorrt_libs"
export LD_LIBRARY_PATH=$TENSOR_LIBS:$CUDNN_PATH/lib:$LD_LIBRARY_PATH

echo $LD_LIBRARY_PATH

xvfb-run python -m alphageometry \
--alsologtostderr \
--problems_file=$(pwd)/imo_ag_30.txt \
--problem_name=translated_imo_2000_p1 \
--mode=ddar \
"${DDAR_ARGS[@]}"

## Run AlphaGeometry

In [ ]:
%%shell

MELIAD_PATH=meliad_lib/meliad
export PYTHONPATH=$PYTHONPATH:/content/alphageometry/$MELIAD_PATH

echo $PYTHONPATH

DATA="/content/drive/MyDrive/ag_ckpt_vocab"

echo $DATA

CUDNN_PATH="/usr/local/lib/python3.10/dist-packages/nvidia/cudnn/"
TENSOR_LIBS="/usr/local/lib/python3.10/dist-packages/tensorrt_libs"
export LD_LIBRARY_PATH=$TENSOR_LIBS:$CUDNN_PATH/lib:$LD_LIBRARY_PATH

echo $LD_LIBRARY_PATH

BATCH_SIZE=2
BEAM_SIZE=2
DEPTH=2

SEARCH_ARGS=(
  --beam_size=$BEAM_SIZE
  --search_depth=$DEPTH
)

LM_ARGS=(
  --ckpt_path=$DATA \
  --vocab_path=$DATA/geometry.757.model \
  --gin_search_paths=$MELIAD_PATH/transformer/configs \
  --gin_file=base_htrans.gin \
  --gin_file=size/medium_150M.gin \
  --gin_file=options/positions_t5.gin \
  --gin_file=options/lr_cosine_decay.gin \
  --gin_file=options/seq_1024_nocache.gin \
  --gin_file=geometry_150M_generate.gin \
  --gin_param=DecoderOnlyLanguageModelGenerate.output_token_losses=True \
  --gin_param=TransformerTaskConfig.batch_size=$BATCH_SIZE \
  --gin_param=TransformerTaskConfig.sequence_length=128 \
  --gin_param=Trainer.restore_state_variables=False
);

xvfb-run python -m alphageometry \
--alsologtostderr \
--problems_file=$(pwd)/examples.txt \
--problem_name=orthocenter \
--mode=alphageometry \
"${DDAR_ARGS[@]}" \
"${SEARCH_ARGS[@]}" \
"${LM_ARGS[@]}"